<a href="https://colab.research.google.com/github/Islander-0v0-wxin/Google-Bert-base-uncased-Training-and-Fine-Tuning/blob/main/Fine_tuning_Google_Bert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset (1).csv


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install -q --upgrade transformers==4.40.2

In [ ]:
!pip install "tensorflow ==2.16.1"
!pip install --upgrade tensorflow-hub
!pip install "tf-models-official==2.16.*"
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import random

In [ ]:
from datasets import load_dataset
import shutil
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import matplotlib.pyplot as plt
import tf_keras as keras
tf.get_logger().setLevel('ERROR')

In [ ]:
!pip show tensorflow
!pip show keras

Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tf-models-official, tf_keras
Name: keras
Version: 3.3.3
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: tensorflow


### Load the dataset

In [ ]:
# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv(file_name)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10460 entries, 0 to 10459
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10460 non-null  object
 1   labels  10460 non-null  int64 
 2   models  10460 non-null  object
dtypes: int64(1), object(2)
memory usage: 245.3+ KB


In [ ]:
df['text'] = df['text'].str.replace('\n', '')

In [ ]:
print(df.isnull().sum()) #check if there is any null data

text      0
labels    0
models    0
dtype: int64


In [ ]:
X = df['text']
y = df['labels']

### Data split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

### Load model and tokenizer

Cased Models: Cased models maintain the original casing of the input text. For example, "Hello" and "hello" would be treated as distinct tokens. These models are trained on text that preserves the original casing.

Uncased Models: Uncased models convert all text to lowercase during tokenization. For example, both "Hello" and "hello" would be tokenized as the same word. These models are trained on text where casing is ignored.

In [ ]:
from transformers import AutoTokenizer, TFBertModel, BertConfig, BertTokenizer
import torch

Bert_hidden_dropout = 0.2
Bert_att_dropout = 0.2

# Configure DistilBERT's initialization
config = BertConfig(hidden_dropout_prob=Bert_att_dropout,
                    attention_probs_dropout_prob=Bert_hidden_dropout,
                    num_labels = 2, # The number of output labels--2 for binary classification.
                    output_attentions = True, # Whether the model returns attentions weights.
                    output_hidden_states=True)

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased", do_lower_case=True, padding = True, pad_token = '[PAD]')
model = TFBertModel.from_pretrained("google-bert/bert-base-uncased", config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
for layer in model.layers:
    layer.trainable = False

### Tokenize text

In [ ]:
MAX_LENGTH = 512


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=16, max_length=MAX_LENGTH):

    input_ids = []
    attention_mask = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='max_length',
                                             #True or 'longest': Pad to the longest sequence in the batch (or no padding if only a single sequence if provided).
                                             #'max_length': Pad to a maximum length specified with the argument max_length or to the maximum acceptable input length for the model if that argument is not provided.
                                             #False or 'do_not_pad' (default): No padding (i.e., can output a batch with sequences of different lengths).
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False,
                                             return_tensors='tf'

                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])


    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)



# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())


In [ ]:
type(X_train_ids)


tensorflow.python.framework.ops.EagerTensor

### Initialize the Base Model

In [ ]:
MAX_LENGTH = 512
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42
batch_size = 16

def build_model(transformer, max_length=MAX_LENGTH):

    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE)

    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length),
                                            batch_size=batch_size,
                                            name='input_ids',dtype=tf.int32)
    input_attention_layer = tf.keras.layers.Input(shape=(max_length),
                                                 batch_size=batch_size,
                                                 name='attention_mask',dtype=tf.int32)

    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
    cls_token = last_hidden_state[:, 0, :]

    ##                                                 ##
    ## Define additional dropout and dense layers here ##
    ##                                                 ##

    # Define a single node that makes up the output layer (for binary classification)
    output = tf.keras.layers.Dense(1, # the number of output classes for classification task
                                   activation='sigmoid',
                                   kernel_initializer=weight_initializer,
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(cls_token)
    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)

    # Compile the model
    model.compile(optimizer= tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE),
                  loss = tf.keras.losses.BinaryCrossentropy(),
                  #run_eagerly=True,
                         metrics= [tf.keras.metrics.BinaryAccuracy(),
                             tf.keras.metrics.FalseNegatives(),
                           tf.keras.metrics.FalsePositives(),
                           tf.keras.metrics.TrueNegatives(),
                           tf.keras.metrics.TruePositives(),
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(),
                           tf.keras.metrics.F1Score(),
                           tf.keras.metrics.AUC(),
                           tf.keras.metrics.AUC(curve='PR'),
                           tf.keras.metrics.AUC(curve='ROC'),
                       ])


    return model

model = build_model(model)

In [ ]:
model.summary()
# Accessing layers attribute
for layer in model.layers:
    print(layer.name)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(16, 512)]                  0         []                            
                                                                                                  
 attention_mask (InputLayer  [(16, 512)]                  0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(16, 512, 768                                       

### Training Classification Layer Weights

In [172]:
y_train_np = np.array(y_train.astype('float32'))
y_test_np = np.array(y_test.astype('float32'))
y_valid_np = np.array(y_valid.astype('float32'))

In [ ]:
EPOCHS = 6
BATCH_SIZE = 16
NUM_STEPS = len(X_train.index) // BATCH_SIZE
#tf.config.run_functions_eagerly(True)

# Train the model
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train_np,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], y_valid_np),
    verbose=2,
    )



Epoch 1/6
457/457 - 105s - loss: 0.4866 - binary_accuracy: 0.8516 - false_negatives: 195.0000 - false_positives: 890.0000 - true_negatives: 2803.0000 - true_positives: 3424.0000 - precision: 0.7937 - recall: 0.9461 - f1_score: 0.6622 - auc: 0.9470 - auc_1: 0.9453 - auc_2: 0.9470 - val_loss: 0.3119 - val_binary_accuracy: 0.9541 - val_false_negatives: 0.0000e+00 - val_false_positives: 72.0000 - val_true_negatives: 693.0000 - val_true_positives: 804.0000 - val_precision: 0.9178 - val_recall: 1.0000 - val_f1_score: 0.6776 - val_auc: 0.9997 - val_auc_1: 0.9997 - val_auc_2: 0.9997 - 105s/epoch - 230ms/step
Epoch 2/6
457/457 - 51s - loss: 0.2753 - binary_accuracy: 0.9786 - false_negatives: 33.0000 - false_positives: 123.0000 - true_negatives: 3559.0000 - true_positives: 3591.0000 - precision: 0.9669 - recall: 0.9909 - f1_score: 0.6631 - auc: 0.9985 - auc_1: 0.9985 - auc_2: 0.9985 - val_loss: 0.1821 - val_binary_accuracy: 0.9771 - val_false_negatives: 0.0000e+00 - val_false_positives: 36.0000 

In [ ]:
model.summary()
# Accessing layers attribute
for layer in model.layers:
    print(layer.name)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(16, 512)]                  0         []                            
                                                                                                  
 attention_mask (InputLayer  [(16, 512)]                  0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(16, 512, 768                                       

### Fine-tuning DistilBERT and Training All Weights

In [ ]:
FT_EPOCHS = 4
BATCH_SIZE = 16
NUM_STEPS = len(X_train.index) // BATCH_SIZE
# tf.config.run_functions_eagerly(True)

# Unfreeze distilBERT layers and make available for training
for layer in model.layers:
    layer.trainable = True

# Recompile model after unfreezing
model.compile(optimizer= tf.keras.optimizers.legacy.Adam(learning_rate=2e-5),
              loss = tf.keras.losses.BinaryCrossentropy(),

              #run_eagerly=True,
              metrics= [tf.keras.metrics.BinaryAccuracy(),
                        tf.keras.metrics.FalseNegatives(),
                           tf.keras.metrics.FalsePositives(),
                           tf.keras.metrics.TrueNegatives(),
                           tf.keras.metrics.TruePositives(),
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(),
                           tf.keras.metrics.F1Score(),
                           tf.keras.metrics.AUC(),
                           tf.keras.metrics.AUC(curve='PR'),
                           tf.keras.metrics.AUC(curve='ROC'),
                       ])



# Save checkpoint
checkpoint_filepath = '/content/drive/My Drive/Research/ckpt/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
#https://keras.io/api/callbacks/model_checkpoint/


# Train the model
train_history2 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train_np,
    epochs = FT_EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], y_valid_np),
    verbose=2,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/4
457/457 - 156s - loss: 0.0076 - binary_accuracy: 0.9978 - false_negatives_1: 10.0000 - false_positives_1: 6.0000 - true_negatives_1: 3688.0000 - true_positives_1: 3608.0000 - precision_1: 0.9983 - recall_1: 0.9972 - f1_score: 0.6620 - auc_3: 0.9997 - auc_4: 0.9996 - auc_5: 0.9997 - val_loss: 0.0090 - val_binary_accuracy: 0.9975 - val_false_negatives_1: 0.0000e+00 - val_false_positives_1: 4.0000 - val_true_negatives_1: 761.0000 - val_true_positives_1: 804.0000 - val_precision_1: 0.9950 - val_recall_1: 1.0000 - val_f1_score: 0.6776 - val_auc_3: 0.9993 - val_auc_4: 0.9988 - val_auc_5: 0.9993 - 156s/epoch - 342ms/step
Epoch 2/4
457/457 - 110s - loss: 0.0038 - binary_accuracy: 0.9992 - false_negatives_1: 4.0000 - false_positives_1: 2.0000 - true_negatives_1: 3680.0000 - true_positives_1: 3620.0000 - precision_1: 0.9994 - recall_1: 0.9989 - f1_score: 0.6631 - auc_3: 0.9999 - auc_4: 0.9997 - auc_5: 0.9999 - val_loss: 0.0025 - val_binary_accuracy: 0.9987 - val_false_negatives_1: 0.00

In [173]:
model.evaluate(x=[X_test_ids, X_test_attention], y=y_test_np, batch_size=16)

99/99 [==============================] - 8s 83ms/step - loss: 7.1798e-04 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives_1: 0.0000e+00 - true_negatives_1: 768.0000 - true_positives_1: 801.0000 - precision_1: 1.0000 - recall_1: 1.0000 - f1_score: 0.6759 - auc_3: 1.0000 - auc_4: 1.0000 - auc_5: 1.0000


[0.0007179828244261444,
 1.0,
 0.0,
 0.0,
 768.0,
 801.0,
 1.0,
 1.0,
 array([0.6759494], dtype=float32),
 0.9999999403953552,
 1.0,
 0.9999999403953552]

In [ ]:
history_df = pd.DataFrame(train_history1.history)
history_df

,loss,binary_accuracy,false_negatives,false_positives,true_negatives,true_positives,precision,recall,f1_score,auc,...,val_false_negatives,val_false_positives,val_true_negatives,val_true_positives,val_precision,val_recall,val_f1_score,val_auc,val_auc_1,val_auc_2
0,0.486621,0.851614,195.0,890.0,2803.0,3424.0,0.793695,0.946118,[0.66215354],0.947010,...,0.0,72.0,693.0,804.0,0.917808,1.0,[0.6776233],0.999693,0.999712,0.999693
1,0.275268,0.978648,33.0,123.0,3559.0,3591.0,0.966882,0.990894,[0.663129],0.998528,...,0.0,36.0,729.0,804.0,0.957143,1.0,[0.6776233],0.999882,0.999890,0.999882
2,0.179126,0.989734,23.0,52.0,3641.0,3590.0,0.985722,0.993634,[0.6617822],0.999438,...,0.0,30.0,735.0,804.0,0.964029,1.0,[0.6776233],0.999901,0.999907,0.999901
3,0.130656,0.991788,25.0,35.0,3657.0,3589.0,0.990342,0.993082,[0.66190475],0.999596,...,0.0,28.0,737.0,804.0,0.966346,1.0,[0.6776233],0.999906,0.999911,0.999906
4,0.099034,0.991651,19.0,42.0,3641.0,3604.0,0.988481,0.994756,[0.6630067],0.999605,...,0.0,24.0,741.0,804.0,0.971014,1.0,[0.6776233],0.999917,0.999922,0.999917
5,0.076736,0.995209,13.0,22.0,3671.0,3600.0,0.993926,0.996402,[0.6617822],0.999746,...,0.0,25.0,740.0,804.0,0.969843,1.0,[0.6776233],0.999928,0.999932,0.999928


In [ ]:
# Save train history1
history_df.to_csv('/content/drive/My Drive/Research/train_history_dense_layer.csv', index=False)


In [ ]:
# save train history2
history_df2 = pd.DataFrame(train_history2.history)
history_df2
history_df2.to_csv('/content/drive/My Drive/Research/train_history2_overall_model.csv', index=False)

### Save model
https://keras.io/api/models/model_saving_apis/model_saving_and_loading/

In [ ]:
## h5 format
model.save("/content/drive/My Drive/Research/bert_base_h5_model.h5")

In [ ]:
## Keras3 format
keras.saving.save_model(model, '/content/drive/My Drive/Research/bert-base-model.keras', overwrite=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
# Define custom objects
custom_objects = {
    'TFBertModel': TFBertModel
}

In [ ]:
filepath = '/content/drive/My Drive/Research/bert-base-tf-model.keras'

In [ ]:
tf.keras.models.save_model(model, filepath)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
loaded_model = tf.keras.models.load_model(filepath, custom_objects=custom_objects, compile=True, safe_mode=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
type(loaded_model)

tf_keras.src.engine.functional.Functional

In [ ]:
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(16, 512)]                  0         []                            
                                                                                                  
 attention_mask (InputLayer  [(16, 512)]                  0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(16, 512, 768                                       

### Evaluate model on new datasets

In [ ]:
from google.colab import files
uploaded1 = files.upload()

Saving train_drcat_04.csv to train_drcat_04.csv


In [ ]:
file_name = list(uploaded1.keys())[0]
df_new = pd.read_csv(file_name)

In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   essay_id  44206 non-null  object
 1   text      44206 non-null  object
 2   label     44206 non-null  int64 
 3   source    44206 non-null  object
 4   prompt    12911 non-null  object
 5   fold      44206 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [ ]:
X_new = df_new['text']
y_new = df_new['label']

In [ ]:
y_new.value_counts()

label
0    29792
1    14414
Name: count, dtype: int64

In [ ]:
y_new.count()

44206

In [ ]:
X_new_test, X_new_valid, y_new_test, y_new_valid = train_test_split(X_new, y_new, test_size=0.5, random_state=42)

In [ ]:
# Encode X__new_train
X_new_train_ids, X_new_train_attention = batch_encode(tokenizer, X_new_test.tolist())

# Encode X__new_valid
X__new_valid_ids, X_new_valid_attention = batch_encode(tokenizer, X_new_valid.tolist())

In [ ]:
y_new_test = np.array(y_new_test.astype('float32'))
y_new_valid = np.array(y_new_valid.astype('float32'))

In [ ]:
# Make predictions on the validation set
y_pred1 = loaded_model.predict([X__new_valid_ids, X_new_valid_attention])

691/691 [==============================] - 110s 145ms/step


In [ ]:
# Make predictions on the test set
y_pred2 = loaded_model.predict([X_new_train_ids, X_new_train_attention])

691/691 [==============================] - 100s 145ms/step


In [ ]:
y_pred1.shape

(22103, 1)

In [ ]:
y_pred_train = (y_pred2 > 0.5).astype(int)
y_pred_val = (y_pred1 > 0.5).astype(int)

In [ ]:
# Evaluate the model on the validation set
val_results = loaded_model.evaluate([X__new_valid_ids, X_new_valid_attention],
                                    y = y_new_valid,
                                    verbose=2,
                                    batch_size=16)

1382/1382 - 110s - loss: 1.1233 - binary_accuracy: 0.7718 - false_negatives_1: 5024.0000 - false_positives_1: 20.0000 - true_negatives_1: 14826.0000 - true_positives_1: 2233.0000 - precision_1: 0.9911 - recall_1: 0.3077 - f1_score: 0.4943 - auc_3: 0.8548 - auc_4: 0.8505 - auc_5: 0.8548 - 110s/epoch - 80ms/step


In [ ]:
test_results = loaded_model.evaluate([X_new_train_ids, X_new_train_attention],
                                      y = y_new_test,
                                      verbose=2,
                                      batch_size = 16)

1382/1382 - 111s - loss: 1.1115 - binary_accuracy: 0.7797 - false_negatives_1: 4858.0000 - false_positives_1: 12.0000 - true_negatives_1: 14934.0000 - true_positives_1: 2299.0000 - precision_1: 0.9948 - recall_1: 0.3212 - f1_score: 0.4892 - auc_3: 0.8529 - auc_4: 0.8495 - auc_5: 0.8529 - 111s/epoch - 80ms/step


In [ ]:
# Get the metric names from the model
metric_names = loaded_model.metrics_names

# Create a DataFrame with the results
results_data = {
    'Metric': metric_names,
    'Test Set': test_results,
    'Validation Set': val_results

}

results_df = pd.DataFrame(results_data)

# Print the DataFrame as a table
print(results_df)


               Metric      Test Set Validation Set
0                loss      1.111464       1.123274
1     binary_accuracy      0.779668       0.771796
2   false_negatives_1        4858.0         5024.0
3   false_positives_1          12.0           20.0
4    true_negatives_1       14934.0        14826.0
5    true_positives_1        2299.0         2233.0
6         precision_1      0.994807       0.991123
7            recall_1      0.321224       0.307703
8            f1_score  [0.48920026]   [0.49434605]
9               auc_3      0.852875       0.854756
10              auc_4      0.849516       0.850545
11              auc_5      0.852875       0.854756


In [176]:
results_df.to_csv('/content/drive/My Drive/Research/results on kaggle essay.csv', index=False)

### Kaggle LLM Detect Dataset

In [183]:
from google.colab import files
uploaded1 = files.upload()

Saving AI_Human.csv to AI_Human.csv


In [195]:
file_name = list(uploaded1.keys())[0]
df2 = pd.read_csv(file_name)

In [199]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487235 entries, 0 to 487234
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   text       487235 non-null  object 
 1   generated  487235 non-null  float64
dtypes: float64(1), object(1)
memory usage: 7.4+ MB


In [200]:
X2 = df2['text']
y2 = df2['generated']

In [201]:
y2.value_counts()

generated
0.0    305797
1.0    181438
Name: count, dtype: int64

In [202]:
X2_test, X2_valid, y2_test, y2_valid = train_test_split(X2, y2, test_size=0.5, random_state=42)

In [203]:
# Encode X__new_train
X2_test_ids, X2_test_attention = batch_encode(tokenizer, X2_test.tolist())

# Encode X__new_valid
X2_valid_ids, X2_valid_attention = batch_encode(tokenizer, X2_valid.tolist())

In [204]:
y2_test = np.array(y2_test.astype('float32'))
y2_valid = np.array(y2_valid.astype('float32'))

In [205]:
test_results1 = loaded_model.evaluate([X2_test_ids, X2_test_attention ],
                                    y = y2_test,
                                    verbose=2,
                                    batch_size=16)

15227/15227 - 1216s - loss: 1.8894 - binary_accuracy: 0.7097 - false_negatives_1: 70623.0000 - false_positives_1: 101.0000 - true_negatives_1: 152846.0000 - true_positives_1: 20047.0000 - precision_1: 0.9950 - recall_1: 0.2211 - f1_score: 0.5425 - auc_3: 0.7669 - auc_4: 0.7777 - auc_5: 0.7669 - 1216s/epoch - 80ms/step


In [206]:
val_results1 = loaded_model.evaluate([X2_valid_ids, X2_valid_attention],
                                    y = y2_valid,
                                    verbose=2,
                                    batch_size=16)

15227/15227 - 1216s - loss: 1.8883 - binary_accuracy: 0.7108 - false_negatives_1: 70351.0000 - false_positives_1: 93.0000 - true_negatives_1: 152757.0000 - true_positives_1: 20417.0000 - precision_1: 0.9955 - recall_1: 0.2249 - f1_score: 0.5429 - auc_3: 0.7665 - auc_4: 0.7772 - auc_5: 0.7665 - 1216s/epoch - 80ms/step


In [207]:
# Get the metric names from the model
metric_names = loaded_model.metrics_names

# Create a DataFrame with the results
results_data1 = {
    'Metric': metric_names,
    'Test Set': test_results1,
    'Validation Set': val_results1

}

results_df1 = pd.DataFrame(results_data1)

# Print the DataFrame as a table
print(results_df1)

               Metric    Test Set Validation Set
0                loss    1.889411       1.888271
1     binary_accuracy    0.709692       0.710842
2   false_negatives_1     70623.0        70351.0
3   false_positives_1       101.0           93.0
4    true_negatives_1    152846.0       152757.0
5    true_positives_1     20047.0        20417.0
6         precision_1    0.994987       0.995466
7            recall_1    0.221098       0.224936
8            f1_score  [0.542468]    [0.5428935]
9               auc_3    0.766885        0.76648
10              auc_4    0.777734       0.777236
11              auc_5    0.766885        0.76648


In [1]:
results_df1.to_csv('/content/drive/My Drive/Research/results on kaggle 400k AI/Human.csv', index=False)

NameError: name 'results_df1' is not defined

### Required Formatting

Special Tokens

[SEP]

At the end of every sentence, we need to append the special [SEP] token.

This token is an artifact of two-sentence tasks, where BERT is given two separate sentences and asked to determine something (e.g., can the answer to the question in sentence A be found in sentence B?).

I am not certain yet why the token is still required when we have only single-sentence input, but it is!

[CLS]

For classification tasks, we must prepend the special [CLS] token to the beginning of every sentence.

This token has special significance. BERT consists of 12 Transformer layers. Each transformer takes in a list of token embeddings, and produces the same number of embeddings on the output (but with the feature values changed, of course!).

###Add a Classification Head

model’s output is a tuple containing:

last_hidden_state → Word-level embedding of shape (batch_size, sequence_length, hidden_size=768).

hidden_states → [Optional] Tuple of tf.Tensor (one for the output of the embeddings + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size=768). Returned when we set

output_hidden_states=True in the config file.
attentions → [Optional] Attention’s weights after the attention softmax, used to compute the weighted average in the self-attention heads. Returned when we set output_attentions=True in the config file.